In [1]:
import pandas as pd

In [3]:
# import libraries for data processing
# -----------------------------------------------------------------------
import pandas as pd

# Import libraries for web browser automation with Selenium
# -----------------------------------------------------------------------
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # ChromeDriverManager manages the installation of the Chrome driver
from selenium.webdriver.common.keys import Keys # to simulate keyboard events in Selenium
from selenium.webdriver.support.ui import Select # to interact with <select> elements on web pages

# import libraries to pause execution
# -----------------------------------------------------------------------
from time import sleep  # Sleep is used to pause the execution of the program for a number of seconds

# settings
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Sets a Pandas option to display all columns of a DataFrame

In [8]:
#Open browser
driver = webdriver.Chrome()

#Variables with Selector to Use
url_cookies = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"
url_search = "#suggestion-search"
url_lookup = "#suggestion-search-button > svg"
url_cast_access = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li > a"
url_birth_year = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(18) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(3) > div > ul > li.ipc-inline-list__item.test-class-react > a"
url_birth_year_exception = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > section > aside > div > span:nth-child(2)"
url_awards = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(2) > a"
url_actor_role = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div > ul"
url_biography = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li > a"
url_init_page = "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div > div.faux-nav > div > a > svg > g:nth-child(1) > rect"
url_init_page_link = "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-e51f9d6d-0.Fmvqg > div > div.ipc-page-content-container.ipc-page-content-container--center.sc-a2225e1f-0.eyUPUI > div > div.sc-a2225e1f-2.PabuC > a"
url_init_page_pro = "#home_img_holder"



# Definición de funciones
def process_input(file_name):
    # open csv file with pandas
    #df = pd.read_csv(f"data/api/{file_name}_API.csv", index_col=0)
    df = pd.read_csv(f"api_comedy_movies/{file_name}_API.csv", index_col=0)

    # transform into a list of tuples
    list_of_movies_from_api = list(df.itertuples(index=True, name=None))

    return list_of_movies_from_api

#combined_results = [] ESTO BORRA LA LISTA!!!

def open_browser(url, driver):
    #Navigate to website
    driver.get(url)
        
    #Maximize window
    driver.maximize_window()
    sleep(5)

def accept_cookies(url_cookies):

    #Accept cookies
    driver.find_element("css selector", url_cookies ).click()
    sleep(5)

def search_movie(url_search, url_lookup, url_cast_access, relation_actor_movie, movie_list):
    full_actor_list = []
    
    for movie_imdb in movie_list:
        print('Película:',movie_imdb)
        
        #Enter search
        search_box = driver.find_element("css selector", url_search)
        #search_box.clear()
        sleep(3)
        search_box.send_keys(movie_imdb[0])
        
        #Click lookup
        driver.find_element("css selector", url_lookup ).click()
        sleep(3)

        #Access cast
        try: 
            driver.find_element("css selector", url_cast_access ).click()
        except Exception as error:
            print("Movie not found")
            sleep(3)
            #driver.find_element("css selector", url_init_page ).click()
            driver.get(url)
            continue
            
        id_imdb = movie_imdb[0]
        actor_list = retrieve_actor_data(url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie)
            
        #Add movie with actor list and info
        full_actor_list.append(actor_list)
        #print("Actors list:", actor_list)

    return full_actor_list

def retrieve_actor_data(url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie):
    
    #Retireve actors
    actor_list = []
   
    try: 
        for i in range(2, 7):
             
            #Actor data
            #Name
            try:
                actor_name = driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a" ).text
            except Exception as error:
                #actor_name = None   
                print("Actor not found") 
                continue
            print("Name:", actor_name)

            #Actor page
            try:
                driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a" ).click()
                sleep(3)
            except Exception as error:
                print("Actor not found")
                continue
            #Birth year
            try:
                actor_birth_year = driver.find_element("css selector", url_birth_year ).text
                actor_birth_year = int(actor_birth_year[-4:])
            except Exception as error:
                try:
                    actor_birth_year = driver.find_element("css selector", url_birth_year_exception ).text
                    actor_birth_year = int(actor_birth_year[-4:])
                except Exception as error:
                    actor_birth_year = None   
            print("Birth year:",actor_birth_year)
            
            #Enter biography
            try:
                driver.find_element("css selector", url_biography ).click()
                sleep(3)
            except Exception as error:
                print("Biography not found")
            
            #Known for
            known_for = []
            try:  
                for j in range(2, 4):
                    known_for.append(driver.find_element("css selector", f"#mini_bio_0 > div > ul > div > div > a:nth-child({j})" ).text)
            except Exception as error:
                known_for.append(None)
            print("Know for:",known_for)

            #Exit biography
            driver.back()
            sleep(2)
            #Actor role
            actor_role = []
        
            try:
                actor_role_detail = driver.find_element("css selector", url_actor_role).text
                detail_with_comma = actor_role_detail.replace("\n", ", ")
                actor_role = detail_with_comma.split(",")
            except Exception as error:
                actor_role.append(None)
            print("Actor role:",actor_role)
            
            #Awards
            actor_awards = []

            try:
                
                driver.find_element("css selector", url_awards ).click()
                sleep(3)
                try:
                    
                    for h in range (3, 5):
                        detail = driver.find_element("css selector", f"#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({h}) > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary").text
                        detail_whit_comma = detail.replace("\n", ", ")
                        actor_awards.append(detail_whit_comma )
                except Exception as error:
                    pass
                #Return to the actor´s page
                driver.back()
                sleep(3)
            except Exception as error:
                actor_awards.append(None)

            if actor_awards == []:
                actor_awards.append(None)
            print("Awards:",actor_awards)     


            clean_actor = clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards)
            clean_actor_list.append(clean_actor)
            print("Actor list for dataframe:",clean_actor_list)
            
            #Add actor information list
            try:
                actor_list.append((actor_name, actor_birth_year, known_for, actor_role, actor_awards))
                print("Actor list:",actor_list)
            except Exception as error:
                print("Error append actor list")

            try:
                relation_actor_movie.append((id_imdb, actor_name))
                print("Relation actor movie:",relation_actor_movie)
            except Exception as error:
                print("Error append relation actor movie")

            driver.back()
            sleep(3)
    except Exception as error:
        pass

    return actor_list
   
def clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards):
    
    clean_know_for = []
    for pelicula in known_for:
        
        if pelicula is None:
            continue
        else:
            partes = pelicula.split(' (')
            # Access each part individually
            titulo = partes[0].replace('"', "")
            # Add the clean title to the new list
            clean_know_for.append(titulo.strip())

    know_for_str = str(clean_know_for).replace("'", "").replace("[", "").replace("]", "") 
    if known_for is None:
        know_for_str = "none"
    
    role_str = str(actor_role).replace("'", "").replace("[", "").replace("]", "")
    if actor_role is None:
        role_str = "none"

    award_str = str(actor_awards).replace("'", "").replace("[", "").replace("]", "")
    if award_str is None:
        award_str = "none"
        
    if actor_birth_year is None:
        actor_birth_year_srt = "none"
    else:
        actor_birth_year_srt = str(actor_birth_year)   

    clean_actor = (actor_name,actor_birth_year_srt,know_for_str,role_str,award_str)
   
    return clean_actor


# Main
file_name = "list_comedy_range_1_2_movie"  
list_of_movies_from_api = process_input(file_name)
relation_actor_movie = []
clean_actor_list = []
url = "https://www.imdb.com/"
open_browser(url, driver)
accept_cookies(url_cookies)
full_actor_list = search_movie(url_search, url_lookup, url_cast_access, relation_actor_movie, list_of_movies_from_api)
print("Relation actor movie",relation_actor_movie)  

# Create DataFrame
headers = ["actor_name", "birth_year", "know_for", "actor_role", "actor_awards"]
df_actor_list = pd.DataFrame(clean_actor_list, columns=headers)
df_actor_list.set_index('actor_name', inplace=True)
df_actor_list



Película: ('Movie', 'Pizzodyssey', 2024, 2, 'tt10408218', 'Comedy')
Name: A.J. Laird
Birth year: None
Know for: ['Through Eyes of Grace', 'Chicago P.D. (2014)']
Actor role: ['Reparto', ' Diseño de producción']
Awards: ['Couch Film Festival', 'Wildwood Film Festival, US']
Through Eyes of Grace
Chicago P.D. (2014)
clean actor ['A.J. Laird', 'none', 'Through Eyes of Grace, Chicago P.D.', 'Reparto,  Diseño de producción', 'Couch Film Festival, Wildwood Film Festival, US']
Lista actores para dataframe: [['A.J. Laird', 'none', 'Through Eyes of Grace, Chicago P.D.', 'Reparto,  Diseño de producción', 'Couch Film Festival, Wildwood Film Festival, US']]
Name: Steve Piper
Birth year: 1969
Know for: ['M3ntal State (2013)', 'Chicago P.D. (2014)']
Actor role: ['Reparto', ' Equipos adicionales']
Awards: ['Couch Film Festival']
M3ntal State (2013)
Chicago P.D. (2014)
clean actor ['Steve Piper', '1969', 'M3ntal State, Chicago P.D.', 'Reparto,  Equipos adicionales', 'Couch Film Festival']
Lista actores 

,birth_year,know_for,actor_role,actor_awards
actor_name,,,,
A.J. Laird,none,"Through Eyes of Grace, Chicago P.D.","Reparto, Diseño de producción","Couch Film Festival, Wildwood Film Festival, US"
Steve Piper,1969,"M3ntal State, Chicago P.D.","Reparto, Equipos adicionales",Couch Film Festival
Catalina Ariel,none,"Where Is She?, Relationships","Reparto, Dirección de arte, Arte",None
Brian Naydol,none,"The Barlow House, Tread This Fantasy","Guion, Dirección, Producción",None
Jada Krueger,none,,Reparto,None
Cailey Fleming,2007,"The Walking Dead, Star Wars: El despertar de l...","Reparto, Banda sonora","CinEuphoria Awards, Blue Whiskey Independent F..."
Ryan Reynolds,1976,"Deadpool, Buried","Reparto, Producción, Guion","Academy of Science Fiction, Fantasy & Horror F..."
John Krasinski,1979,"Un lugar tranquilo, The Office","Reparto, Producción, Dirección","Academy of Science Fiction, Fantasy & Horror F..."
Fiona Shaw,1958,"Harry Potter y la piedra filosofal, Los vengad...","Reparto, Equipos adicionales, Banda sonora","BAFTA Awards, British Independent Film Awards"


In [ ]:
# save into a .csv file
df_actor_list.to_csv(f"data/selenium_actors/{file_name}_sel_actor.csv")

In [9]:
# # Create DataFrame
# headers = ["id_imdb", "actor_name"]
# df_actor_movie_list = pd.DataFrame(relation_actor_movie, columns=headers)
# df_actor_movie_list.set_index('id_imdb', inplace=True)
# df_actor_movie_list

,actor_name
id_imdb,
tt10408218,A.J. Laird
tt10408218,Steve Piper
tt10408218,Catalina Ariel
tt10408218,Brian Naydol
tt10408218,Jada Krueger
tt11152168,Cailey Fleming
tt11152168,Ryan Reynolds
tt11152168,John Krasinski
tt11152168,Fiona Shaw


In [ ]:
# #Open browser
# driver = webdriver.Chrome()

# #Variables with Selector to Use
# url_cookies = "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"
# url_search = "#suggestion-search"
# url_lookup = "#suggestion-search-button > svg"
# url_cast_access = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li > a"
# url_birth_year = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(18) > div.sc-f65f65be-0.bBlII > ul > li:nth-child(3) > div > ul > li.ipc-inline-list__item.test-class-react > a"
# url_birth_year_exception = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-11.cvvyMK > section > aside > div > span:nth-child(2)"
# url_awards = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li:nth-child(2) > a"
# url_actor_role = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-491663c0-3.bdjVSf > div > ul"
# url_biography = "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > section > div:nth-child(5) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > ul > li > a"


# #combined_results = [] ESTO BORRA LA LISTA!!!

# def open_browser(url, driver):
#     #Navigate to website
#     driver.get(url)
        
#     #Maximize window
#     driver.maximize_window()
#     sleep(3)

# def accept_cookies(url_cookies):

#     #Accept cookies
#     driver.find_element("css selector", url_cookies ).click()
#     sleep(3)

# def search_movie(url_search, url_lookup, url_cast_access, relation_actor_movie, movie_list):
#     full_actor_list = []
    
#     for movie_imdb in movie_list[:2]:
#         print('Película:',movie_imdb)
        
#         #Enter search
#         driver.find_element("css selector", url_search).send_keys(movie_imdb[4])
    
#         #Click lookup
#         driver.find_element("css selector", url_lookup ).click()
        
#         sleep(5)
#         #Access cast
#         driver.find_element("css selector", url_cast_access ).click()
#         id_imdb = movie_imdb[4]
#         actor_list = retrieve_actor_data( url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie)
            
#         #Add movie with actor list and info
#         full_actor_list.append(actor_list)
        
#         print("Actors list:", actor_list)

#     return full_actor_list

# def retrieve_actor_data(url_birth_year, url_biography, url_actor_role, url_awards, id_imdb, relation_actor_movie):
    
#     #Retireve actors
#     actor_list = []
   
#     try: 
#         for i in range(2, 7):
                        
#             #Actor data
#             #Name
#             try:
#                 actor_name = driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a" ).text
#             except Exception as error:
#                 actor_name = None    
#             print("Name:", actor_name)

#             #Actor page
#             try:
#                 driver.find_element("css selector", f"#fullcredits_content > table.cast_list > tbody > tr:nth-child({i}) > td:nth-child(2) > a" ).click()
#             except Exception as error:
#                 print("Actor not found")
                
#             #Birth year
#             try:
#                 actor_birth_year = driver.find_element("css selector", url_birth_year ).text
#                 actor_birth_year = int(actor_birth_year[-4:])
#             except Exception as error:
#                 try:
#                     actor_birth_year = driver.find_element("css selector", url_birth_year_exception ).text
#                     actor_birth_year = int(actor_birth_year[-4:])
#                 except Exception as error:
#                     actor_birth_year = None   
#             print("Birth year:",actor_birth_year)
            
#             #Biography
#             try:
#                 driver.find_element("css selector", url_biography ).click()
#             except Exception as error:
#                 print("Biography not found")
            
#             #Known for
#             known_for = []
#             try:  
#                 for j in range(2,4):
#                     known_for.append(driver.find_element("css selector", f"#mini_bio_0 > div > ul > div > div > a:nth-child({j})" ).text)
#             except Exception as error:
#                 known_for.append(None)
#             print("Know for:",known_for)

#             #Exit biography
#             driver.back()
            
#             #Actor role
#             actor_role = []
        
#             try:
#                 actor_role_detail = driver.find_element("css selector", url_actor_role).text
#                 detail_with_comma = actor_role_detail.replace("\n", ", ")
#                 actor_role = detail_with_comma.split(",")
                
#             except Exception as error:
#                 actor_role.append(None)
#             print("Actor role:",actor_role)
            
#             #Awards
#             actor_awards = []

#             try:
                
#                 driver.find_element("css selector", url_awards ).click()
#                 try:
                    
#                     for h in range (3, 5):
#                         detail = driver.find_element("css selector", f"#__next > main > div > section > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child({h}) > div.ipc-title.ipc-title--base.ipc-title--section-title.ipc-title--on-textPrimary").text
#                         detail_whit_comma = detail.replace("\n", ", ")
#                         actor_awards.append(detail_whit_comma )
#                 except Exception as error:
#                     pass
#                 #Return to the actor´s page
#                 driver.back()
#             except Exception as error:
#                 actor_awards.append(None)

#             if actor_awards == []:
#                 actor_awards.append(None)
#             print("Awards:",actor_awards)     


#             clean_actor = clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards)
#             clean_actor_list.append(clean_actor)
#             print("Lista actores para dataframe:",clean_actor_list)
#             #Add actor information list
#             actor_list.append((actor_name, actor_birth_year, known_for, actor_role, actor_awards))

#             relation_actor_movie.append([id_imdb, actor_name])

#             driver.back()
#     except Exception as error:
#         pass

#     return actor_list
   
# def clean_year_know_for(actor_name, actor_birth_year, known_for, actor_role, actor_awards):
    
#     clean_know_for = []
#     for pelicula in known_for:
#         print(pelicula)
#         if pelicula is None:
#             continue
#         else:
#             partes = pelicula.split(' (')
#             # Access each part individually
#             titulo = partes[0].replace('"', "")
#             # Add the clean title to the new list
#             clean_know_for.append(titulo.strip())

#     know_for_str = str(clean_know_for).replace("'", "").replace("[", "").replace("]", "") 
#     if known_for is None:
#         know_for_str = "none"
    
#     role_str = str(actor_role).replace("'", "").replace("[", "").replace("]", "")
#     if actor_role is None:
#         role_str = "none"

#     award_str = str(actor_awards).replace("'", "").replace("[", "").replace("]", "")
#     if award_str is None:
#         award_str = "none"
        
#     if actor_birth_year is None:
#         actor_birth_year_srt = "none"
#     else:
#         actor_birth_year_srt = str(actor_birth_year)   

#     clean_actor = [actor_name,actor_birth_year_srt,know_for_str,role_str,award_str]
#     print("clean actor",clean_actor)
#     return clean_actor


# # Main
# relation_actor_movie = []
# clean_actor_list = []
# url = "https://www.imdb.com/"
# open_browser(url, driver)
# accept_cookies(url_cookies)
# full_actor_list = search_movie(url_search, url_lookup, url_cast_access, relation_actor_movie, combined_results)
# print("Relation actor movie",relation_actor_movie)  

# # Create DataFrame
# headers = ["actor_name", "birth_year", "know_for", "actor_role", "actor_awards"]
# df_actor_list = pd.DataFrame(clean_actor_list, columns=headers)
# df_actor_list.set_index('actor_name', inplace=True)
# df_actor_list
